In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
init_printing()

In [2]:
p = symbols('p:3', real=True)
v = symbols('v:3', real=True)
f = symbols('f:3', real=True)
m = symbols('m', real=True)
t = symbols('t', real=True)
o = symbols('o:7', real=True)

In [3]:
a = [f_/m for f_ in f]
model = Matrix([
    m,
    p[0] + v[0]*t + (a[0]*t**2)/2,
    p[1] + v[1]*t + (a[1]*t**2)/2,
    p[2] + v[2]*t + (a[2]*t**2)/2,
    v[0] + a[0]*t,
    v[1] + a[1]*t,
    v[2] + a[2]*t,
])
model

⎡        m        ⎤
⎢                 ⎥
⎢    2            ⎥
⎢f₀⋅t             ⎥
⎢───── + p₀ + t⋅v₀⎥
⎢ 2⋅m             ⎥
⎢                 ⎥
⎢    2            ⎥
⎢f₁⋅t             ⎥
⎢───── + p₁ + t⋅v₁⎥
⎢ 2⋅m             ⎥
⎢                 ⎥
⎢    2            ⎥
⎢f₂⋅t             ⎥
⎢───── + p₂ + t⋅v₂⎥
⎢ 2⋅m             ⎥
⎢                 ⎥
⎢    f₀⋅t         ⎥
⎢    ──── + v₀    ⎥
⎢     m           ⎥
⎢                 ⎥
⎢    f₁⋅t         ⎥
⎢    ──── + v₁    ⎥
⎢     m           ⎥
⎢                 ⎥
⎢    f₂⋅t         ⎥
⎢    ──── + v₂    ⎥
⎣     m           ⎦

In [4]:
J_model = model.jacobian((m,*p,*v))
J_model

⎡   1     0  0  0  0  0  0⎤
⎢                         ⎥
⎢     2                   ⎥
⎢-f₀⋅t                    ⎥
⎢───────  1  0  0  t  0  0⎥
⎢     2                   ⎥
⎢  2⋅m                    ⎥
⎢                         ⎥
⎢     2                   ⎥
⎢-f₁⋅t                    ⎥
⎢───────  0  1  0  0  t  0⎥
⎢     2                   ⎥
⎢  2⋅m                    ⎥
⎢                         ⎥
⎢     2                   ⎥
⎢-f₂⋅t                    ⎥
⎢───────  0  0  1  0  0  t⎥
⎢     2                   ⎥
⎢  2⋅m                    ⎥
⎢                         ⎥
⎢-f₀⋅t                    ⎥
⎢──────   0  0  0  1  0  0⎥
⎢   2                     ⎥
⎢  m                      ⎥
⎢                         ⎥
⎢-f₁⋅t                    ⎥
⎢──────   0  0  0  0  1  0⎥
⎢   2                     ⎥
⎢  m                      ⎥
⎢                         ⎥
⎢-f₂⋅t                    ⎥
⎢──────   0  0  0  0  0  1⎥
⎢   2                     ⎥
⎣  m                      ⎦

In [5]:
observe = Matrix(o[1:4])
observe

⎡o₁⎤
⎢  ⎥
⎢o₂⎥
⎢  ⎥
⎣o₃⎦

In [6]:
J_observe = observe.jacobian(o)
J_observe

⎡0  1  0  0  0  0  0⎤
⎢                   ⎥
⎢0  0  1  0  0  0  0⎥
⎢                   ⎥
⎣0  0  0  1  0  0  0⎦

In [7]:
lambda_model = lambdify((m,*p,*v,*f,t), model)
lambda_J_model = lambdify((m,*p,*v,*f,t), J_model)
lambda_observe = lambdify(o, observe)
lambda_J_observe = lambdify(o, J_observe)

In [8]:
def ekf(state, covar, control, measure, q=1e-3, r=1e-3):
    predict_state = lambda_model(*state[:,0], *control[:,0])
    F = lambda_J_model(*state[:,0], *control[:,0])
    Q = np.random.randn(*[F.shape[0]]*2) * q
    predict_covar = F @ covar @ F.T + Q
    residual_state = measure - lambda_observe(*predict_state[:,0])
    H = lambda_J_observe(*predict_state[:,0])
    R = np.random.randn(*[H.shape[0]]*2) * r
    PHt = predict_covar @ H.T
    residual_covar = H @ PHt + R
    try:
        kalman_gain = PHt @ np.linalg.inv(residual_covar)
    except:
        return state, covar
    updated_state = predict_state + kalman_gain @ residual_state
    updated_covar = (np.identity(F.shape[0]) - (kalman_gain @ H)) @ predict_covar
    return updated_state, updated_covar

In [9]:
state_real = np.random.randn(7)[:,np.newaxis] * 10
state_real[0,0] = abs(state_real[0,0])
state_esti = np.array([
    state_real[0,0]+np.random.randn(),
    0,0,0,0,0,0])[:,np.newaxis]
covar_esti = np.identity(state_esti.shape[0]) * 1e6
control = np.array([0,0,0,1e-2])[:,np.newaxis]
for i in range(1000):
    control[:3,0] += np.random.randn(3)
    state_real = lambda_model(*state_real[:,0], *control[:,0])
    measure = lambda_observe(*state_real[:,0])
    state_esti, covar_esti = ekf(state_esti, covar_esti, control, measure)
print(np.concatenate([state_real, state_esti], axis=-1))

[[ 28.37565505  28.35791858]
 [ 55.46434947  55.46413675]
 [-37.19863681 -37.19838833]
 [152.37760334 152.37738228]
 [ -0.49258137  -0.49718244]
 [ -5.90229759  -5.90608685]
 [ 13.69754961  13.69809415]]
